In [1]:
from exps.math_rlvr import RLVRMathDataset, to_datum
import rich

/Users/dipakkrishnan/git/clones/tuning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = RLVRMathDataset("Qwen/Qwen3-8B")

##  Sample item

In [3]:
item = dataset[0]

In [4]:
item.keys()

dict_keys(['prompt_tokens', 'model_input', 'ground_truth', 'messages'])

In [5]:
rich.print(item["messages"])

[
    {
        'role': 'system',
        'content': 'You are a helpful math assistant. Think through the problem carefully in <think> tags, then 
provide your final answer in \\boxed{}. /think'
    },
    {
        'content': 'Question: Find the domain of the expression $\\frac{\\sqrt{x-2}}{\\sqrt{5-x}}$.}\nAnswer:The 
expressions inside each square root must be non-negative.\nTherefore, $x-2 \\ge 0$, so $x\\ge2$, and $5 - x \\ge 
0$, so $x \\le 5$.\nAlso, the denominator cannot be equal to zero, so $5-x>0$, which gives $x<5$.\nTherefore, the 
domain of the expression is $\\boxed{[2,5)}$.\n\nQuestion: If $\\det \\mathbf{A} = 2$ and $\\det \\mathbf{B} = 12,$
then find $\\det (\\mathbf{A} \\mathbf{B}).$\nAnswer:We have that $\\det (\\mathbf{A} \\mathbf{B}) = (\\det 
\\mathbf{A})(\\det \\mathbf{B}) = (2)(12) = \\boxed{24}.$\n\nQuestion: Terrell usually lifts two 20-pound weights 
12 times. If he uses two 15-pound weights instead, how many times must Terrell lift them in order to lift the same 
total weight?\nAnswer:If Terrell lifts two 20-pound weights 12 times, he lifts a total of $2\\cdot 12\\cdot20=480$ 
pounds of weight.  If he lifts two 15-pound weights instead for $n$ times, he will lift a total of $2\\cdot15\\cdot
n=30n$ pounds of weight.  Equating this to 480 pounds, we can solve for $n$: 
\\begin{align*}\n30n&=480\\\\\n\\Rightarrow\\qquad n&=480/30=\\boxed{16}\n\\end{align*}\n\nQuestion: If the system 
of equations\n\n\\begin{align*}\n6x-4y&=a,\\\\\n6y-9x &=b.\n\\end{align*}has a solution $(x, y)$ where $x$ and $y$ 
are both nonzero, find $\\frac{a}{b},$ assuming $b$ is nonzero.\nAnswer:If we multiply the first equation by 
$-\\frac{3}{2}$, we obtain\n\n$$6y-9x=-\\frac{3}{2}a.$$Since we also know that $6y-9x=b$, we 
have\n\n$$-\\frac{3}{2}a=b\\Rightarrow\\frac{a}{b}=\\boxed{-\\frac{2}{3}}.$$\n\nQuestion: What is the modulo $13$ 
residue of $247+5 \\cdot 39 + 7 \\cdot 143 +4 \\cdot 15?$',
        'role': 'user'
    }
]

In [6]:
rich.print(f"Ground truth: {item['ground_truth']}")

Ground truth: 8

In [7]:
rich.print(f"Prompt tokens: {len(item['prompt_tokens'])}")

Prompt tokens: 642

## Messages Format

This format for each question is typical of how you prompt models in this RLVR environment. You give examples in context and then have the model mimic the format of those examples. We'll compute reward as a composite of format reward and correctness.

In [10]:
rich.print(f"Ground truth: {item['ground_truth']}")
rich.print(f"Prompt tokens: {len(item['prompt_tokens'])}")

Ground truth: 8

Prompt tokens: 642

## Inference on Sample Question

Let's see what Qwen-8B can do on this question.

In [11]:
from exps.math_rlvr.utils import create_clients, get_sampling_params
from exps.math_rlvr.models import TrainingConfig, SamplingConfig

In [12]:
train_config = TrainingConfig()
sampling_config = SamplingConfig()

In [13]:
clients = create_clients(
    config=train_config
)

In [16]:
future = clients.sampling_client.sample(
    prompt=item["model_input"], 
    sampling_params=get_sampling_params(sampling_config), 
    num_samples=1
)

In [17]:
response = future.result()

In [19]:
for i, sequence in enumerate(response.sequences):
    decoded_text = dataset.tokenizer.decode(sequence.tokens)
    print(f"Sample {i+1}: {decoded_text}")

Sample 1: <think>
Okay, so I need to find the modulo 13 residue of the expression 247 + 5*39 + 7*143 + 4*15. Hmm, modulo 13 residue means I have to compute the entire expression and then find the remainder when divided by 13. But maybe there's a smarter way than calculating each term first and then adding them up. Let me think.

First, I remember that when dealing with modulo operations, I can take each term modulo 13 first and then add them up, right? Because modulo is distributive over addition and multiplication. So maybe I can simplify each term individually before adding. Let me try that.

Starting with the first term: 247. Let me divide 247 by 13 and find the remainder. Let me recall that 13*19 is 247? Wait, 13*20 is 260, so 260 - 13 = 247, so 13*19 = 247. Oh, so 247 divided by 13 is exactly 19 with no remainder. Therefore, 247 mod 13 is 0. Okay, that's straightforward.

Next term is 5*39. Let me compute 5*39 first. 5*39 is 195. Now, find 195 mod 13. Alternatively, maybe I can fi

## Notes

Yes, the model got the answer right and formatted it properly. But it shows that the task is within reach and reward won't be too sparse. The task needs to be achievable! In this case, Qwen8B has been sort of "warm started" with SFT to produce traces that contain the <think> tags.